In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import hmean
from scipy.stats import norm 

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scikitplot.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from time import time
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [3]:
import pandas as pd
df = pd.read_csv("../input/clean-16-million-tweets/clean_tweets_again.csv")

In [4]:
df.dropna(inplace = True)
df.drop(columns = 'Unnamed: 0',inplace = True)

In [5]:
X = df['text']
y = df['sentiment']

In [6]:
# the data is split in 98|1|1
from sklearn.model_selection import train_test_split
X_train,X_validation_test,y_train,y_validation_test = train_test_split(X,y,test_size = 0.02,random_state = 45)

In [7]:
X_validation,X_test,y_validation,y_test = train_test_split(X_validation_test,y_validation_test,test_size = 0.5,random_state = 45)

In [8]:
#in previous parts we calculated the harmonic mean of positive CDF and negative CDF which happen to show good representation of each class
#more over this both features sucessfully filters which terms are important to each class and which terms are not
count_vec = CountVectorizer(max_features = 10000)
count_vec.fit(X_train)#in this case we are creating a bag of words of 10,000 features from the dataset

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=10000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [9]:
neg_train = X_train[y_train == 0] #this are the values from training set where where sentiment is negative
pos_train = X_train[y_train == 1] # this are the values from training set where sentiment is positive
negative_document_matrix = count_vec.transform(neg_train) # once we get the positive and negative tweets we transform the positive tweets into a sparse matrix
positive_document_matrix = count_vec.transform(pos_train) 

In [10]:
neg_tf = np.sum(negative_document_matrix,axis = 0) #in this case we are finding the sum of frequencies of negative features
pos_tf = np.sum(positive_document_matrix,axis = 0) #in this case we are finding the sum of frequencies of positive features

In [11]:
from scipy.stats import hmean
from scipy.stats import norm
def norm_cdf(x):
    return norm.cdf(x,x.mean(),x.std())
neg = np.squeeze(np.array(neg_tf))
pos = np.squeeze(np.array(pos_tf))
term_freq_df = pd.DataFrame([neg,pos],columns = count_vec.get_feature_names()).T

In [12]:
term_freq_df.columns = ['positive','negative']
term_freq_df['total'] = term_freq_df['positive'] + term_freq_df['negative']
term_freq_df['pos_rate'] = term_freq_df['positive'] / term_freq_df['total']
term_freq_df['pos_freq_pct'] = term_freq_df['positive'] / term_freq_df['positive'].sum()
term_freq_df['pos_rate_cdf'] = norm_cdf(term_freq_df['pos_rate'])
term_freq_df['pos_freq_pct_cdf'] = norm_cdf(term_freq_df['pos_freq_pct'])
term_freq_df['pos_hmean_cdf'] = hmean([term_freq_df['pos_rate_cdf'],term_freq_df['pos_freq_pct_cdf']])

In [13]:
pos_hmean = term_freq_df['pos_hmean_cdf']

In [14]:
y_predict_probablity = []
for text in X_test:
    h_mean_score = [pos_hmean[word] for word in text.split() if word in pos_hmean.index]
    if len(h_mean_score) > 0:
        pred_proba = np.mean(h_mean_score)
    else:
        pred_proba = np.random.random()
    y_predict_probablity.append(pred_proba)
pred = [1 if val > 0.50 else 0 for val in y_predict_probablity]

In [15]:
accuracy_score(y_test,pred)

0.3696510243719065